In [ ]:
!pip install stellargraph
# !pip install tf-nightly

     |████████████████████████████████| 435 kB 4.3 MB/s 
     |████████████████████████████████| 462 kB 78.4 MB/s 


In [ ]:
import stellargraph as sg
import os
import sys
import urllib.request

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.python.ops import array_ops
#om stellargraph.layer import GCN_LSTM

import datetime
import random
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/EzPC/gcn_gcn-lstm

Mounted at /content/gdrive
/content/gdrive/MyDrive/EzPC/gcn_gcn-lstm


In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import activations, initializers, constraints, regularizers
from tensorflow.keras.layers import Input, Layer, Dropout, LSTM, Dense, Permute, Reshape
from stellargraph.mapper import SlidingFeaturesNodeGenerator
from stellargraph.core.experimental import experimental
from stellargraph.core.utils import calculate_laplacian

In [ ]:
hinitial = tf.random.normal([1, 16])
cinitial = tf.random.normal([1, 16])

#Run

In [ ]:
from timeit import default_timer as timer

class TimingCallback(keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

In [ ]:
def data_preparation(datafile):
    #Put the file location
    df = pd.read_csv(datafile)
    
    #type casting
    df.pm1_0 = df.pm1_0.astype(float)
    df.pm2_5 = df.pm2_5.astype(float)
    df.pm10 = df.pm10.astype(float)
    df.lat = round(round(5*df.lat.astype(float),2)/5.0,3)
    df.long= round(round(5*df.long.astype(float),2)/5.0,3)

    # Ensuring Delhi region and removing outliers from data
    df = df[(df.lat.astype(int) == 28) &(df.long.astype(int) == 77)]
    df = df[(df.pm1_0<=1500) & (df.pm2_5<=1500) & (df.pm10<=1500) & (df.pm1_0>=20) & (df.pm2_5>=30) & (df.pm10>=30)]
    #df = df[(df.humidity<=60)&(df.humidity>=7)]
    # TODO: Add buffer and consider train-data for range
    # lat_range = {'min': df.lat.min(), 'max': df.lat.max()}
    # long_range = {'min': df.long.min(), 'max': df.long.max()}
    df['lat_grid'] = df.apply(lambda row: int((n_lat_grid-1)*(row.lat-lat_range['min'])/(lat_range['max']-lat_range['min'])), axis=1 )
    df['long_grid'] = df.apply(lambda row: int((n_long_grid-1)*(row.long-long_range['min'])/(long_range['max']-long_range['min'])), axis=1 )
    df['lat_grid'] = df['lat_grid'].astype(float).astype(int)
    df['long_grid'] = df['long_grid'].astype(float).astype(int)
    means = df[["lat_grid", "long_grid", "pm2_5"]].groupby(['lat_grid', 'long_grid']).mean().reset_index()['pm2_5'].values

    # rounding @120min
    df.dateTime = pd.to_datetime(df.dateTime)
    df.dateTime = df.dateTime.dt.round('120min')
    # use time as a feature as well
    df.dateTime = df.dateTime.dt.hour*60 + df.dateTime.dt.minute

    # only PM2_5
    df = df[['dateTime','lat_grid','long_grid','pm2_5']]
    df = df.pivot_table(index=['lat_grid','long_grid'], columns='dateTime', aggfunc='mean')['pm2_5']
    df = df.fillna(0)
    df['means'] = means
    return df

In [ ]:
def my_loss(y_true, y_pred):
    mse = tf.keras.losses.MeanSquaredError()
    #mask = tf.sign(y_true)
    #y_pred = y_pred*(mask)
    loss= y_true-y_pred
    #loss = mse(tf.reshape(y_true,(-1,)),tf.reshape(y_pred,(-1,)))
#     print(loss)
    return loss

In [ ]:
def train_test_split(df, grid_points_indices, train_size):
    random.seed(10)
#     train_size = 0.8
    #train-data for range from total grid structure
    points_len = df.shape[0]
    test_size = int(points_len * (1-train_size))
    test_indices = random.sample(list(df.index), test_size)
    train_indices = list(set(df.index) - set(test_indices))

    train_data = df.loc[train_indices, :] 
    train_data = train_data.reindex(grid_points_indices,fill_value=0)
    train_data = np.array(train_data)

    test_data = df.loc[test_indices, :] 
    test_data = test_data.reindex(grid_points_indices,fill_value=0)
    test_data = np.array(test_data)
    return train_data, test_data

In [ ]:
def scale_data(train_data, test_data):
    max_pm = train_data.max()
    min_pm = train_data.min()
    train_scaled = (train_data - min_pm) / (max_pm - min_pm)
    test_scaled = (test_data - min_pm) / (max_pm - min_pm)
    return train_scaled, test_scaled

In [ ]:
def sequence_data_preparation(seq_len, train_data, test_data):
    trainX, trainY, testX, testY = [], [], [], []

    trainX.append(train_data[:, 0 : 0 + seq_len])
    trainY.append(train_data[:, -1])
    testX.append(test_data[:, 0 : 0 + seq_len])
    testY.append(test_data[:, -1])
    trainX = np.array(trainX)
    trainY = np.array(trainY)
    testX = np.array(testX)
    testY = np.array(testY)

    return trainX, trainY, testX, testY

In [ ]:
def prepare_train_data(df, grid_points_indices, train_size):
    train_data, test_data = train_test_split(df, grid_points_indices, train_size)
    print("Train data: ", train_data.shape)
    print("Test data: ", test_data.shape)
    train_scaled, test_scaled = scale_data(train_data, test_data)
    trainX, trainY, testX, testY = sequence_data_preparation(
        seq_len, train_scaled, test_scaled
    )
    print(trainX.shape)
    print(trainY.shape)
    print(testX.shape)
    print(testY.shape)
    return trainX, trainY, testX, testY, train_data

In [ ]:
def train_model(trainX, trainY, testX, testY, train_data):
    cb = TimingCallback()

    history = model.fit(
            trainX,
            trainY,
            epochs = 100,
            batch_size = 40,
            shuffle=True,
            verbose=0,
            validation_data=[trainX, testY],
            callbacks=[cb]
        )

    ## Rescale values
    max_pm = train_data.max()
    min_pm = train_data.min()
    train_scaled = (train_data - min_pm) / (max_pm - min_pm)
    
    print(
        "Mean Train loss: ",
        sum(tf.sqrt(history.history["loss"]))*max_pm/len(history.history["loss"]),
        "\n Last Train loss: ",
        tf.sqrt(history.history["loss"][-1])*max_pm,
        "\n Mean Test loss:",
        sum(tf.sqrt(history.history["val_loss"]))*max_pm/len(history.history["val_loss"]),
        "\n Last Test loss:",
        tf.sqrt(history.history["val_loss"][-1])*max_pm,
    #     "\nTrain time ", traintime,
        "\n Train time callbacks", sum(cb.logs)
        
    )
    sg.utils.plot_history(history)

In [ ]:
def eval(model, trainX, testY):
    output = model.predict(trainX)
    # yhat = model.predict(testX)

    ## actual train and test values
    ## Rescale values
    max_pm = train_data.max()
    min_pm = train_data.min()
    print(max_pm)
    train_true = np.array((trainY * (max_pm - min_pm)) + min_pm)
    test_true = np.array((testY * (max_pm - min_pm)) + min_pm)

    ## Rescale model predicted values
    train_output = np.array((output * (max_pm - min_pm)) + min_pm)
    # test_rescpred = np.array((yhat * (max_pm - min_pm)) + min_pm)

    # # Masked predicted values
    # mask_train = tf.sign(trainY)
    mask_test = tf.sign(testY)
    # train_rescpred = train_rescpred*(mask_train)
    test_output = train_output*(mask_test)
    test_mse = my_loss(test_true, train_output)
    test_rmse = tf.sqrt(test_mse)
    print("Test RMSE: ", test_rmse)
    return test_output, test_true

In [ ]:
from scipy.spatial.distance import pdist, squareform

lat_range = {'min': 28.486, 'max': 28.72}
long_range = {'min': 77.1, 'max': 77.32}
n_lat_grid = 100
n_long_grid = 100
seq_len = 12

#adjacency matrix
grid_points = pd.read_csv('grid_points.csv')
n_grid_points = grid_points.shape[0] 
X = grid_points[["lat_grid", "long_grid"]].to_numpy()
f = lambda u, v: max(abs(u-v))<=1
adj = squareform(pdist(X, f))
eye = np.identity(n_grid_points)
adj = adj+eye
print(adj.shape) 

grid_points = grid_points.pivot_table(index=['lat_grid', 'long_grid'])
grid_points_indices = grid_points.index

(1989, 1989)


#Model

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import activations, initializers, constraints, regularizers
from tensorflow.keras.layers import Input, Layer, Dropout, LSTM, Dense, Permute, Reshape
from stellargraph.mapper import SlidingFeaturesNodeGenerator
from stellargraph.core.experimental import experimental
from stellargraph.core.utils import calculate_laplacian
class FixedAdjacencyGraphConvolution(Layer):

    """
    Graph Convolution (GCN) Keras layer.
    The implementation is based on https://github.com/tkipf/keras-gcn.
    Original paper: Semi-Supervised Classification with Graph Convolutional Networks. Thomas N. Kipf, Max Welling,
    International Conference on Learning Representations (ICLR), 2017 https://github.com/tkipf/gcn
    Notes:
      - The inputs are 3 dimensional tensors: batch size, sequence length, and number of nodes.
      - This class assumes that a simple unweighted or weighted adjacency matrix is passed to it,
        the normalized Laplacian matrix is calculated within the class.
    Args:
        units (int): dimensionality of output feature vectors
        A (N x N): weighted/unweighted adjacency matrix
        activation (str or func): nonlinear activation applied to layer's output to obtain output features
        use_bias (bool): toggles an optional bias
        kernel_initializer (str or func, optional): The initialiser to use for the weights.
        kernel_regularizer (str or func, optional): The regulariser to use for the weights.
        kernel_constraint (str or func, optional): The constraint to use for the weights.
        bias_initializer (str or func, optional): The initialiser to use for the bias.
        bias_regularizer (str or func, optional): The regulariser to use for the bias.
        bias_constraint (str or func, optional): The constraint to use for the bias.
    """

    def __init__(
        self,
        units,
        A,
        activation=None,
        use_bias=True,
        input_dim=None,
        kernel_initializer="glorot_uniform",
        kernel_regularizer=None,
        kernel_constraint=None,
        bias_initializer="zeros",
        bias_regularizer=None,
        bias_constraint=None,
        **kwargs,
    ):
        if "input_shape" not in kwargs and input_dim is not None:
            kwargs["input_shape"] = (input_dim,)

        self.units = units
        self.adj = calculate_laplacian(A) if A is not None else None

        self.activation = activations.get(activation)
        self.use_bias = use_bias

        self.kernel_initializer = initializers.get(kernel_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_initializer = initializers.get(bias_initializer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.bias_constraint = constraints.get(bias_constraint)

        super().__init__(**kwargs)

    def get_config(self):
        """
        Gets class configuration for Keras serialization.
        Used by Keras model serialization.
        Returns:
            A dictionary that contains the config of the layer
        """

        config = {
            "units": self.units,
            "use_bias": self.use_bias,
            "activation": activations.serialize(self.activation),
            "kernel_initializer": initializers.serialize(self.kernel_initializer),
            "kernel_regularizer": regularizers.serialize(self.kernel_regularizer),
            "kernel_constraint": constraints.serialize(self.kernel_constraint),
            "bias_initializer": initializers.serialize(self.bias_initializer),
            "bias_regularizer": regularizers.serialize(self.bias_regularizer),
            "bias_constraint": constraints.serialize(self.bias_constraint),
            # the adjacency matrix argument is required, but
            # (semi-secretly) supports None for loading from a saved
            # model, where the adjacency matrix is a saved weight
            "A": None,
        }

        base_config = super().get_config()
        return {**base_config, **config}

    def compute_output_shape(self, input_shapes):
        """
        Computes the output shape of the layer.
        Assumes the following inputs:
        Args:
            input_shapes (tuple of int)
                Shape tuples can include None for free dimensions, instead of an integer.
        Returns:
            An input shape tuple.
        """
        feature_shape = input_shapes

        return feature_shape[0], feature_shape[1], self.units

    def build(self, input_shapes):
        """
        Builds the layer
        Args:
            input_shapes (list of int): shapes of the layer's inputs (the batches of node features)
        """
        _batch_dim, n_nodes, features = input_shapes

        if self.adj is not None:
            adj_init = initializers.constant(self.adj)
        else:
            adj_init = initializers.zeros()

        self.A = self.add_weight(
            name="A", shape=(n_nodes, n_nodes), trainable=False, initializer=adj_init
        )
        self.kernel = self.add_weight(
            shape=(features, self.units),
            initializer=self.kernel_initializer,
            name="kernel",
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
        )

        if self.use_bias:
            self.bias = self.add_weight(
                # ensure the per-node bias can be broadcast across each feature
                shape=(n_nodes, 1),
                initializer=self.bias_initializer,
                name="bias",
                regularizer=self.bias_regularizer,
                constraint=self.bias_constraint,
            )
        else:
            self.bias = None
        self.built = True

    def call(self, features):
        """
        Applies the layer.
        Args:
            features (ndarray): node features (size B x N x F), where B is the batch size, F = TV is
                the feature size (consisting of the sequence length and the number of variates), and
                N is the number of nodes in the graph.
        Returns:
            Keras Tensor that represents the output of the layer.
        """

        # Calculate the layer operation of GCN
        # shape = B x F x N
        nodes_last = tf.transpose(features, [0, 2, 1])
        neighbours = K.dot(nodes_last, self.A)
        
        # shape = B x N x F
        h_graph = tf.transpose(neighbours, [0, 2, 1])
        # shape = B x N x units
        output = K.dot(h_graph, self.kernel)

        # Add optional bias & apply activation
        if self.bias is not None:
            output += self.bias
        #relus
        output = self.activation(output)

        return output


@experimental(
    reason="Lack of unit tests and code refinement", issues=[1132, 1526, 1564]
)
class GCN_LSTM:

    """
    GCN_LSTM is a univariate timeseries forecasting method. The architecture  comprises of a stack of N1 Graph Convolutional layers followed by N2 LSTM layers, a Dropout layer, and  a Dense layer.
    This main components of GNN architecture is inspired by: T-GCN: A Temporal Graph Convolutional Network for Traffic Prediction (https://arxiv.org/abs/1811.05320).
    The implementation of the above paper is based on one graph convolution layer stacked with a GRU layer.
    The StellarGraph implementation is built as a stack of the following set of layers:
    1. User specified no. of Graph Convolutional layers
    2. User specified no. of LSTM layers
    3. 1 Dense layer
    4. 1 Dropout layer.
    The last two layers consistently showed better performance and regularization experimentally.
    .. seealso::
       Example using GCN_LSTM: `spatio-temporal time-series prediction <https://stellargraph.readthedocs.io/en/stable/demos/time-series/gcn-lstm-time-series.html>`__.
       Appropriate data generator: :class:`.SlidingFeaturesNodeGenerator`.
       Related model: :class:`.GCN` for graphs without time-series node features.
    Args:
       seq_len: No. of LSTM cells
       adj: unweighted/weighted adjacency matrix of [no.of nodes by no. of nodes dimension
       gc_layer_sizes (list of int): Output sizes of Graph Convolution  layers in the stack.
       lstm_layer_sizes (list of int): Output sizes of LSTM layers in the stack.
       generator (SlidingFeaturesNodeGenerator): A generator instance.
       bias (bool): If True, a bias vector is learnt for each layer in the GCN model.
       dropout (float): Dropout rate applied to input features of each GCN layer.
       gc_activations (list of str or func): Activations applied to each layer's output; defaults to ``['relu', ..., 'relu']``.
       lstm_activations (list of str or func): Activations applied to each layer's output; defaults to ``['tanh', ..., 'tanh']``.
       kernel_initializer (str or func, optional): The initialiser to use for the weights of each layer.
       kernel_regularizer (str or func, optional): The regulariser to use for the weights of each layer.
       kernel_constraint (str or func, optional): The constraint to use for the weights of each layer.
       bias_initializer (str or func, optional): The initialiser to use for the bias of each layer.
       bias_regularizer (str or func, optional): The regulariser to use for the bias of each layer.
       bias_constraint (str or func, optional): The constraint to use for the bias of each layer.
     """

    def __init__(
        self,
        seq_len,
        adj,
        gc_layer_sizes,
        lstm_layer_sizes,
        gc_activations=None,
        generator=None,
        lstm_activations=None,
        bias=True,
        dropout=0.5,
        kernel_initializer=None,
        kernel_regularizer=None,
        kernel_constraint=None,
        bias_initializer=None,
        bias_regularizer=None,
        bias_constraint=None,
    ):
        if generator is not None:
            if not isinstance(generator, SlidingFeaturesNodeGenerator):
                raise ValueError(
                    f"generator: expected a SlidingFeaturesNodeGenerator, found {type(generator).__name__}"
                )

            if seq_len is not None or adj is not None:
                raise ValueError(
                    "expected only one of generator and (seq_len, adj) to be specified, found multiple"
                )

            adj = generator.graph.to_adjacency_matrix(weighted=True).todense()
            seq_len = generator.window_size
            variates = generator.variates
        else:
            variates = None

        super(GCN_LSTM, self).__init__()

        n_gc_layers = len(gc_layer_sizes)
        n_lstm_layers = len(lstm_layer_sizes)

        self.lstm_layer_sizes = lstm_layer_sizes
        self.gc_layer_sizes = gc_layer_sizes
        self.bias = bias
        self.dropout = dropout
        self.adj = adj
        self.n_nodes = adj.shape[0]
        self.n_features = seq_len
        self.seq_len = seq_len
        self.multivariate_input = variates is not None
        self.variates = variates if self.multivariate_input else 1
        self.outputs = self.n_nodes * self.variates

        self.kernel_initializer = initializers.get(kernel_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_initializer = initializers.get(bias_initializer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.bias_constraint = constraints.get(bias_constraint)

        # Activation function for each gcn layer
        if gc_activations is None:
            gc_activations = ["relu"] * n_gc_layers
        elif len(gc_activations) != n_gc_layers:
            raise ValueError(
                "Invalid number of activations; require one function per graph convolution layer"
            )
        self.gc_activations = gc_activations

        # Activation function for each lstm layer
        if lstm_activations is None:
            lstm_activations = ["tanh"] * n_lstm_layers
        elif len(lstm_activations) != n_lstm_layers:
            padding_size = n_lstm_layers - len(lstm_activations)
            if padding_size > 0:
                lstm_activations = lstm_activations + ["tanh"] * padding_size
            else:
                raise ValueError(
                    "Invalid number of activations; require one function per lstm layer"
                )
        self.lstm_activations = lstm_activations

        self._gc_layers = [
            FixedAdjacencyGraphConvolution(
                units=self.variates * layer_size,
                A=self.adj,
                activation=activation,
                kernel_initializer=self.kernel_initializer,
                kernel_regularizer=self.kernel_regularizer,
                kernel_constraint=self.kernel_constraint,
                bias_initializer=self.bias_initializer,
                bias_regularizer=self.bias_regularizer,
                bias_constraint=self.bias_constraint,
            )
            for layer_size, activation in zip(self.gc_layer_sizes, self.gc_activations)
        ]
        self._lstm_layers = [
            LSTM(layer_size, activation=activation, return_sequences=True)
            for layer_size, activation in zip(
                self.lstm_layer_sizes[:-1], self.lstm_activations
            )
        ]
        self._lstm_layers.append(
            LSTM(
                self.lstm_layer_sizes[-1],
                activation=self.lstm_activations[-1],
                return_sequences=False,
            )
        )
        self._decoder_layer = Dense(self.outputs, activation="sigmoid")

    def __call__(self, x):

        x_in, out_indices = x

        h_layer = x_in
        if not self.multivariate_input:
            # normalize to always have a final variate dimension, with V = 1 if it doesn't exist
            # shape = B x N x T x 1
            h_layer = tf.expand_dims(h_layer, axis=-1)

        # flatten variates into sequences, for convolution
        # shape B x N x (TV)
        h_layer = Reshape((self.n_nodes, self.seq_len * self.variates))(h_layer)

        for layer in self._gc_layers:
            print("GCN", layer)
            h_layer = layer(h_layer)

        
        # return the layer to its natural multivariate tensor form
        # shape B x N x T' x V (where T' is the sequence length of the last GC)
        h_layer = Reshape((self.n_nodes, -1, self.variates))(h_layer)
        # put time dimension first for LSTM layers
        # shape B x T' x N x V
        
        h_layer = Permute((2, 1, 3))(h_layer)
        
        # flatten the variates across all nodes, shape B x T' x (N V)
        h_layer = Reshape((-1, self.n_nodes * self.variates))(h_layer)
        
        for layer in self._lstm_layers:
            h_layer = layer(h_layer, initial_state=[hinitial, cinitial])

        
        #h_layer = Dropout(self.dropout)(h_layer)"""
        h_layer = self._decoder_layer(h_layer)

        if self.multivariate_input:
            # flatten things out to the multivariate shape
            # shape B x N x V
            h_layer = Reshape((self.n_nodes, self.variates))(h_layer)

        return h_layer

    def in_out_tensors(self):
        """
        Builds a GCN model for node  feature prediction
        Returns:
            tuple: ``(x_inp, x_out)``, where ``x_inp`` is a list of Keras/TensorFlow
                input tensors for the GCN model and ``x_out`` is a tensor of the GCN model output.
        """
        # Inputs for features
        if self.multivariate_input:
            shape = (None, self.n_nodes, self.n_features, self.variates)
        else:
            shape = (None, self.n_nodes, self.n_features)

        x_t = Input(batch_shape=shape)

        # Indices to gather for model output
        out_indices_t = Input(batch_shape=(None, self.n_nodes), dtype="int32")

        x_inp = [x_t, out_indices_t]
        x_out = self(x_inp)

        return x_inp[0], x_out

In [ ]:
def my_loss(y_true, y_pred):
    mse = tf.keras.losses.MeanSquaredError()
    #mask = tf.sign(y_true)
    #y_pred = y_pred*(mask)
    loss= mse(y_true, y_pred)
    #loss = mse(tf.reshape(y_true,(-1,)),tf.reshape(y_pred,(-1,)))
#     print(loss)
    return loss

In [ ]:
initializer = tf.keras.initializers.Zeros()
values = initializer(shape=(2, 2))

In [ ]:
print(values)

tf.Tensor(
[[0. 0.]
 [0. 0.]], shape=(2, 2), dtype=float32)


In [ ]:
hinitial = tf.random.normal([1, 16])
cinitial = tf.random.normal([1, 16])

In [ ]:
"""gcn_lstm = GCN_LSTM(
        seq_len=seq_len,
        adj=adj,
        gc_layer_sizes=[4, 4],
        gc_activations=["relu", "relu"],
        lstm_layer_sizes=[4, 4],
        lstm_activations=["tanh", "tanh"],
    )
x_input, x_output = gcn_lstm.in_out_tensors()
print(x_input)
print(x_output)
model = Model(inputs=x_input, outputs=x_output)
#opt = tf.optimizers.Adam(learning_rate = 0.005)
#opt=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False, name='SGD')
model.compile(optimizer='sgd', loss=my_loss, metrics=["MSE"], run_eagerly=True)
print(model.summary())"""

In [ ]:
gcn_lstm = GCN_LSTM(
        seq_len=seq_len,
        adj=adj,
        gc_layer_sizes=[16, 16],
        gc_activations=["relu", "relu"],
        lstm_layer_sizes=[16],
        lstm_activations=["tanh"],
    )
x_input, x_output = gcn_lstm.in_out_tensors()
print(x_input)
print(x_output)
model = Model(inputs=x_input, outputs=x_output)
#opt = tf.optimizers.Adam(learning_rate = 0.005)
#opt=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False, name='SGD')
model.compile(optimizer='sgd', loss=my_loss, metrics=["MSE"], run_eagerly=True)
print(model.summary())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ExperimentalWarning: GCN_LSTM is experimental: Lack of unit tests and code refinement (see: https://github.com/stellargraph/stellargraph/issues/1132, https://github.com/stellargraph/stellargraph/issues/1526, https://github.com/stellargraph/stellargraph/issues/1564). It may be difficult to use and may have major changes at any time.
  import sys


GCN <__main__.FixedAdjacencyGraphConvolution object at 0x7f4039f2c950>
GCN <__main__.FixedAdjacencyGraphConvolution object at 0x7f403a006710>
KerasTensor(type_spec=TensorSpec(shape=(None, 1989, 12), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")
KerasTensor(type_spec=TensorSpec(shape=(1, 1989), dtype=tf.float32, name=None), name='dense/Sigmoid:0', description="created by layer 'dense'")
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1989, 12)]        0         
                                                                 
 tf.expand_dims (TFOpLambda)  (None, 1989, 12, 1)      0         
                                                                 
 reshape (Reshape)           (None, 1989, 12)          0         
                                                                 
 fixed_adjacency_grap

In [ ]:
print(calculate_laplacian(adj))

[[0.25       0.20412415 0.         ... 0.         0.         0.        ]
 [0.20412415 0.16666667 0.16666667 ... 0.         0.         0.        ]
 [0.         0.16666667 0.16666667 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.5        0.         0.        ]
 [0.         0.         0.         ... 0.         0.33333333 0.28867513]
 [0.         0.         0.         ... 0.         0.28867513 0.25      ]]


In [ ]:
print(model.weights[5])

<tf.Variable 'fixed_adjacency_graph_convolution_1/A:0' shape=(1989, 1989) dtype=float32, numpy=
array([[0.25      , 0.20412415, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.20412415, 0.16666667, 0.16666667, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.16666667, 0.16666667, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.5       , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.33333334,
        0.28867513],
       [0.        , 0.        , 0.        , ..., 0.        , 0.28867513,
        0.25      ]], dtype=float32)>


In [ ]:
for i in model.weights:
  print(i.name, i.shape)

fixed_adjacency_graph_convolution_8/kernel:0 (12, 16)
fixed_adjacency_graph_convolution_8/bias:0 (1989, 1)
fixed_adjacency_graph_convolution_8/A:0 (1989, 1989)
fixed_adjacency_graph_convolution_9/kernel:0 (16, 16)
fixed_adjacency_graph_convolution_9/bias:0 (1989, 1)
fixed_adjacency_graph_convolution_9/A:0 (1989, 1989)
lstm_6/lstm_cell_6/kernel:0 (1989, 64)
lstm_6/lstm_cell_6/recurrent_kernel:0 (16, 64)
lstm_6/lstm_cell_6/bias:0 (64,)
dense_4/kernel:0 (16, 1989)
dense_4/bias:0 (1989,)


#Forward pass

In [ ]:
#put adj into a list (1989, 1989)
num=16
inputs = tf.random.normal([1, 1989, 12])

In [ ]:
for i in range(len(model.weights)):
  print(i, model.weights[i].name, model.weights[i].shape)

0 fixed_adjacency_graph_convolution_8/kernel:0 (12, 16)
1 fixed_adjacency_graph_convolution_8/bias:0 (1989, 1)
2 fixed_adjacency_graph_convolution_8/A:0 (1989, 1989)
3 fixed_adjacency_graph_convolution_9/kernel:0 (16, 16)
4 fixed_adjacency_graph_convolution_9/bias:0 (1989, 1)
5 fixed_adjacency_graph_convolution_9/A:0 (1989, 1989)
6 lstm_6/lstm_cell_6/kernel:0 (1989, 64)
7 lstm_6/lstm_cell_6/recurrent_kernel:0 (16, 64)
8 lstm_6/lstm_cell_6/bias:0 (64,)
9 dense_4/kernel:0 (16, 1989)
10 dense_4/bias:0 (1989,)


In [ ]:
for i in relu_outputs:
  print(i.shape)

(1, 1989, 16)
(1, 1989, 16)


In [ ]:
fixed_layers_inputs=[]
relu_outputs=[]
a=tf.transpose(inputs, [0, 2, 1])
neighbours=K.dot(a, model.weights[2])

fixed_layers_inputs.append(a)
h_graph = tf.transpose(neighbours, [0, 2, 1])
output = K.dot(h_graph, model.weights[0])

fixed_layers_inputs.append(h_graph)
output += model.weights[1]
relu_outputs.append(output)
output = tf.keras.activations.relu(output)

a=output

neighbours=K.dot(tf.transpose(a, [0, 2, 1]), model.weights[5])
fixed_layers_inputs.append(a)
h_graph = tf.transpose(neighbours, [0, 2, 1])
output = K.dot(h_graph, model.weights[3])
fixed_layers_inputs.append(h_graph)
output += model.weights[4]
relu_outputs.append(output)
output = tf.keras.activations.relu(output)
output = tf.transpose(output, [0, 2, 1])

c=cinitial
h=hinitial
cache=[]
cache2=[]
inputs_np=[]
cache.append((h, c, 0, 0, 0, 0, 0))
for i in range(16):
    inp=tf.constant([output[0][i].numpy()])
    inputs_np.append(inp)
    z = K.dot(inp, model.weights[6])
    z += K.dot(h, model.weights[7])
    z = K.bias_add(z, model.weights[8])
    z0, z1, z2, z3 = array_ops.split(z, 4, axis=1)

    i = tf.sigmoid(z0)
    f = tf.sigmoid(z1)
    c = f * c + i * tf.tanh(z2)
    o = tf.sigmoid(z3)
    h = o * tf.tanh(c)
    cache.append((h, c, i, f, tf.tanh(z2), o, inp))
print(h)
"""
c=cinitial
h=hinitial
for i in range(4):
    inp=cache[i+1][0]
    z = K.dot(inp, model.weights[9])
    z += K.dot(h, model.weights[10])
    z = K.bias_add(z, model.weights[11])
    z0, z1, z2, z3 = array_ops.split(z, 4, axis=1)

    i = tf.sigmoid(z0)
    f = tf.sigmoid(z1)
    c = f * c + i * tf.tanh(z2)
    o = tf.sigmoid(z3)
    h = o * tf.tanh(c)

    cache2.append((h, c, i, f, tf.tanh(z2), o, inp))"""


tf.Tensor(
[[ 0.17200643 -0.07490341  0.20572524  0.05792527  0.11283293  0.2694692
  -0.19968204 -0.03091206  0.23499937  0.18030389  0.08836638  0.11205715
   0.15205424 -0.12135664 -0.1215778   0.10481273]], shape=(1, 16), dtype=float32)


'\nc=cinitial\nh=hinitial\nfor i in range(4):\n    inp=cache[i+1][0]\n    z = K.dot(inp, model.weights[9])\n    z += K.dot(h, model.weights[10])\n    z = K.bias_add(z, model.weights[11])\n    z0, z1, z2, z3 = array_ops.split(z, 4, axis=1)\n\n    i = tf.sigmoid(z0)\n    f = tf.sigmoid(z1)\n    c = f * c + i * tf.tanh(z2)\n    o = tf.sigmoid(z3)\n    h = o * tf.tanh(c)\n\n    cache2.append((h, c, i, f, tf.tanh(z2), o, inp))'

In [ ]:
print(model(inputs))
print(tf.sigmoid(K.dot(h, model.weights[9])[0].numpy()))

tf.Tensor([[0.50251985 0.507687   0.49927709 ... 0.49897808 0.49560565 0.4947646 ]], shape=(1, 1989), dtype=float32)
tf.Tensor([0.50251985 0.507687   0.49927709 ... 0.49897808 0.49560565 0.4947646 ], shape=(1989,), dtype=float32)


In [ ]:
print(model(inputs))
print(tf.sigmoid(K.dot(h, model.weights[12])[0].numpy()))

In [ ]:
#labels = tf.random.normal([1, 1989])
weights_copy_9 = tf.Variable(model.weights[9])
weights_copy_6 = tf.Variable(model.weights[6])
weights_copy_7 = tf.Variable(model.weights[7])
outputs_=model(inputs)

#Backward

In [ ]:
def relu_derivative(x):
  relu_deriv=[]
  for i in x.numpy():
    l=[]
    for j in i:
      if(j>0):
        l.append(1.0)
      else:
        l.append(0.0)
    relu_deriv.append(l)
  return convert_to_tensor(relu_deriv)

In [ ]:
#Sigmoid
"""
print(deriv/(weights_copy_9-model.weights[9]))
model.fit(inputs, labels, epochs = 1, batch_size = 1, shuffle=False, steps_per_epoch=1)
print(deriv/(weights_copy_9-model.weights[9]))"""

'\nprint(deriv/(weights_copy_9-model.weights[9]))\nmodel.fit(inputs, labels, epochs = 1, batch_size = 1, shuffle=False, steps_per_epoch=1)\nprint(deriv/(weights_copy_9-model.weights[9]))'

In [ ]:
print(weights_copy_9)

<tf.Variable 'Variable:0' shape=(16, 1989) dtype=float32, numpy=
array([[ 0.02477845,  0.03853293, -0.03118273, ...,  0.0184625 ,
         0.00278362, -0.04764629],
       [ 0.00983692, -0.04856486,  0.00379442, ...,  0.05257556,
         0.04941424,  0.03317813],
       [-0.00844471,  0.01939088, -0.01536951, ...,  0.01636462,
         0.00597642, -0.01040287],
       ...,
       [ 0.04683192, -0.03646107,  0.04972583, ..., -0.00350509,
        -0.01577148,  0.04452905],
       [ 0.02968954, -0.00019949, -0.0464611 , ...,  0.0161799 ,
        -0.02655751,  0.00250624],
       [ 0.03299098, -0.01834105, -0.01736215, ...,  0.00117147,
         0.0348537 , -0.00045371]], dtype=float32)>


In [ ]:
deriv=(labels-outputs_)

deriv=deriv*(outputs_)*(1-outputs_)
dL_dh=K.dot(deriv, tf.transpose(weights_copy_9))
deriv=K.dot(tf.transpose(h), deriv)
print(deriv)
deriv=tf.squeeze(deriv)
print(dL_dh)

tf.Tensor(
[[-0.0609269   0.0500498  -0.05302665 ...  0.03467407 -0.02638139
  -0.01284803]
 [ 0.02653176 -0.02179512  0.02309144 ... -0.01509947  0.01148827
   0.00559492]
 [-0.07287054  0.05986117 -0.06342158 ...  0.04147131 -0.03155299
  -0.01536666]
 ...
 [ 0.04298609 -0.03531191  0.03741218 ... -0.02446379  0.018613
   0.00906474]
 [ 0.04306443 -0.03537626  0.03748036 ... -0.02450837  0.01864692
   0.00908126]
 [-0.03712602  0.03049802 -0.03231198 ...  0.02112877 -0.01607559
  -0.00782899]], shape=(16, 1989), dtype=float32)
tf.Tensor(
[[-0.05121101  0.03179879  0.11343273  0.11613809 -0.08761348  0.39369535
  -0.16574287 -0.24688298 -0.0278164   0.07988106 -0.0630185  -0.31611344
  -0.01764496  0.33126947 -0.20774384 -0.4851143 ]], shape=(1, 16), dtype=float32)


In [ ]:
for i in range(16):
  h, c, i, f, g, o, x=cache[16-i]
  print(g)

tf.Tensor(
[[ 0.12722956 -0.097331    0.01735084 -0.07359414  0.06893248  0.08817872
   0.03444234 -0.19850366  0.02628486  0.04219221  0.18917072  0.1239162
   0.09184273  0.06517559  0.11673123  0.23487763]], shape=(1, 16), dtype=float32)
tf.Tensor(
[[ 0.35279042 -0.04424164  0.07397331  0.15133393  0.20134188  0.32075518
  -0.227867    0.08358616  0.29420337  0.49962118  0.15160494  0.3509523
   0.47555533 -0.19422553 -0.37384346 -0.25963098]], shape=(1, 16), dtype=float32)
tf.Tensor(
[[ 0.21766642 -0.13353363  0.6054381   0.2553335   0.20633762  0.68400663
  -0.79522645  0.10542493  0.6004991   0.24494359  0.17615001  0.10566177
   0.46521446 -0.6203948  -0.4842483   0.23915087]], shape=(1, 16), dtype=float32)
tf.Tensor(
[[ 0.21011822 -0.2071704   0.16534153 -0.05494312 -0.05797583  0.11038879
  -0.20114881  0.02959221  0.32787645 -0.16662484  0.03137236  0.21570016
   0.1535314  -0.02415966  0.1132468   0.3041046 ]], shape=(1, 16), dtype=float32)
tf.Tensor(
[[ 0.05500823 -0.084311

In [ ]:
dh=dL_dh
dgates=[]
dct_1=0
ft_1=0
dX=[]

for index in range(16):
  ind=num-index

  h, c, i, f, g, o, x=cache[ind]
  cprev=cache[ind-1][1]

  dct=dh*o*(1-pow(tf.tanh(c),2)) + dct_1*ft_1
  dct_1=dct
  ft_1=f

  dgt=dct*i*(1-pow(g,2))
  dit=dct*g*i*(1-i)
  dft=dct*cprev*f*(1-f)
  dot=dh*tf.tanh(c)*o*(1-o)

  dgate=tf.concat([dit, dft, dgt, dot], 1)
  dXt=K.dot(dgate, tf.transpose(model.weights[6]))
  print(dXt)
  dX.append(dXt)
  dh=K.dot(dgate, tf.transpose(model.weights[7]))
  dgates.append(dgate)

dgates.reverse()
dX.reverse()

tf.Tensor(
[[ 0.00872015 -0.00107904 -0.00884407 ... -0.003313   -0.00278453
  -0.00091195]], shape=(1, 1989), dtype=float32)
tf.Tensor(
[[ 0.00263589  0.00109446 -0.0027332  ... -0.00077167 -0.00020738
  -0.00161595]], shape=(1, 1989), dtype=float32)
tf.Tensor(
[[ 0.00314694  0.00062597 -0.00186546 ... -0.00223647  0.00054065
  -0.00023044]], shape=(1, 1989), dtype=float32)
tf.Tensor(
[[ 0.00266653 -0.00026975 -0.00137796 ... -0.00158492 -0.0001625
  -0.00084685]], shape=(1, 1989), dtype=float32)
tf.Tensor(
[[ 1.6944133e-03  3.3111442e-05 -9.4767334e-04 ... -1.1576003e-03
  -4.4984629e-04 -9.5440494e-04]], shape=(1, 1989), dtype=float32)
tf.Tensor(
[[ 1.1636922e-03  2.8159766e-04 -7.6517853e-04 ... -1.2228223e-03
  -2.7492370e-06 -8.5055688e-04]], shape=(1, 1989), dtype=float32)
tf.Tensor(
[[ 8.7312667e-04  2.6010830e-04 -8.8401568e-05 ... -9.0307457e-04
   3.1372238e-04 -1.0026308e-03]], shape=(1, 1989), dtype=float32)
tf.Tensor(
[[ 0.00068187  0.00015221 -0.00016347 ... -0.00088304 

In [ ]:
from tensorflow.python.framework.ops import convert_to_tensor
dX=convert_to_tensor(dX)
dX=tf.squeeze(dX)

In [ ]:
print(dX)

tf.Tensor(
[[-1.1013985e-04  5.7709465e-05  2.3525159e-05 ... -1.9909498e-04
   2.7024240e-04 -3.7295534e-04]
 [-4.0988569e-05  5.7573397e-05  1.6419187e-06 ... -3.1757279e-04
   2.1637832e-04 -3.3821748e-04]
 [ 1.8610372e-05  1.5174640e-05 -1.8853892e-07 ... -3.1829119e-04
   2.5972846e-04 -4.2910941e-04]
 ...
 [ 3.1469432e-03  6.2596577e-04 -1.8654622e-03 ... -2.2364731e-03
   5.4065336e-04 -2.3044270e-04]
 [ 2.6358925e-03  1.0944583e-03 -2.7331994e-03 ... -7.7167351e-04
  -2.0737926e-04 -1.6159480e-03]
 [ 8.7201465e-03 -1.0790399e-03 -8.8440739e-03 ... -3.3130047e-03
  -2.7845304e-03 -9.1195444e-04]], shape=(16, 1989), dtype=float32)


In [ ]:
dW=tf.zeros((1989, num*4))
dU=tf.zeros((num, num*4))
for i in range(num):
  xnext=inputs_np[i]
  hnext=cache[i][0]
  dW+=K.dot(tf.transpose(xnext), dgates[i])
  dU+=K.dot(tf.transpose(hnext), dgates[i])

In [ ]:
for i in fixed_layers_inputs:
  print(i.shape)
for i in model.weights:
  print(i.name, i.shape)

(1, 12, 1989)
(1, 1989, 12)
(1, 1989, 16)
(1, 1989, 16)
fixed_adjacency_graph_convolution_8/kernel:0 (12, 16)
fixed_adjacency_graph_convolution_8/bias:0 (1989, 1)
fixed_adjacency_graph_convolution_8/A:0 (1989, 1989)
fixed_adjacency_graph_convolution_9/kernel:0 (16, 16)
fixed_adjacency_graph_convolution_9/bias:0 (1989, 1)
fixed_adjacency_graph_convolution_9/A:0 (1989, 1989)
lstm_6/lstm_cell_6/kernel:0 (1989, 64)
lstm_6/lstm_cell_6/recurrent_kernel:0 (16, 64)
lstm_6/lstm_cell_6/bias:0 (64,)
dense_4/kernel:0 (16, 1989)
dense_4/bias:0 (1989,)


In [ ]:
1989, 1989
12, 16
1989, 1

In [ ]:
r1=tf.transpose(tf.squeeze(tf.Variable(relu_outputs[1])))

In [ ]:
#Relu derivative
#relu_deriv=[1 else 0 for i in dX if (i>0)]print(tf.transpose(tf.squeeze(dy_dx))/dgcnw11)
relu_1=relu_derivative(r1)
drelu_1=dX*relu_1
print(drelu_1)
#Weight 1 derivative
temp=tf.squeeze(fixed_layers_inputs[3])
dgcnw11=K.dot(tf.transpose(temp), tf.transpose(drelu_1))
print(dgcnw11.shape)

#Input 1 Derivative
dgcn_11=K.dot(model.weights[3], drelu_1)
print(dgcn_11.shape)

#Weight 2 derivative
temp=tf.squeeze(fixed_layers_inputs[2])
dgcnw12=K.dot(temp, dgcn_11)
print(dgcnw12.shape)

#Input 2 derivative
dgcn_12=K.dot(dgcn_11, model.weights[5]) #next weight set is 0, 2
print(dgcn_12.shape)

tf.Tensor(
[[-1.1013985e-04  5.7709465e-05  2.3525159e-05 ... -1.9909498e-04
   2.7024240e-04 -3.7295534e-04]
 [-4.0988569e-05  5.7573397e-05  1.6419187e-06 ... -0.0000000e+00
   2.1637832e-04 -3.3821748e-04]
 [ 0.0000000e+00  0.0000000e+00 -1.8853892e-07 ... -3.1829119e-04
   2.5972846e-04 -4.2910941e-04]
 ...
 [ 3.1469432e-03  6.2596577e-04 -1.8654622e-03 ... -2.2364731e-03
   0.0000000e+00 -0.0000000e+00]
 [ 2.6358925e-03  1.0944583e-03 -2.7331994e-03 ... -0.0000000e+00
  -2.0737926e-04 -1.6159480e-03]
 [ 0.0000000e+00 -0.0000000e+00 -0.0000000e+00 ... -0.0000000e+00
  -0.0000000e+00 -0.0000000e+00]], shape=(16, 1989), dtype=float32)
(16, 16)
(16, 1989)
(1989, 1989)
(16, 1989)


In [ ]:
print(dgcnw12)

tf.Tensor(
[[ 3.5446985e-03  1.0489768e-03 -2.6042811e-03 ... -1.7249557e-03
  -6.5743268e-05 -1.0034104e-03]
 [ 2.9483596e-03  7.0380815e-04 -1.8838024e-03 ... -1.6034305e-03
  -3.1670606e-05 -6.0037186e-04]
 [ 4.1583850e-04  3.0404516e-04 -4.9114192e-04 ... -3.1779002e-04
   1.6922777e-04 -7.0568000e-04]
 ...
 [ 1.7309883e-03 -4.3996470e-04  5.7911268e-05 ... -4.9276152e-03
   5.7444657e-04  6.9950742e-04]
 [-7.7084178e-04  5.8549829e-04 -5.9519330e-04 ...  1.2949362e-03
   5.4305291e-04 -3.0398611e-03]
 [-2.8391066e-04  6.7047682e-04 -7.2941347e-04 ...  7.3652982e-04
   4.6847915e-04 -2.6789850e-03]], shape=(1989, 1989), dtype=float32)


In [ ]:
r2=tf.transpose(tf.squeeze(tf.Variable(relu_outputs[0])))

In [ ]:
 #Relu Derivative
#Change what the relu derivative is multiplied by
drelu_2=dgcn_12*relu_derivative(r2)

#Weight 1 derivative for the second layer
temp=tf.squeeze(fixed_layers_inputs[1])
dgcnw_21=K.dot(drelu_2, temp)
print(dgcnw_21.shape)

#Input 1 derivative for the second layer 
dgcn_21=K.dot(model.weights[0], drelu_2)
print(dgcn_21.shape)

#Weight 2 derivative for the second layer
dgcnw_22=tf.squeeze(K.dot(tf.transpose(fixed_layers_inputs[0], [0, 2, 1]), dgcn_21))
print(dgcnw_22.shape)
for i in range(16):
  print((dgcnw_22)[i])

(16, 12)
(12, 1989)
(1989, 1989)
tf.Tensor(
[ 0.00863328  0.0043025  -0.0006814  ...  0.00128458 -0.00014729
 -0.0002004 ], shape=(1989,), dtype=float32)
tf.Tensor(
[-9.9471351e-04 -4.3336858e-04  2.5104471e-05 ...  1.2572741e-04
 -1.7300213e-04 -1.6173517e-04], shape=(1989,), dtype=float32)
tf.Tensor(
[-0.00136349 -0.0004765   0.0004239  ... -0.0005739  -0.00035774
 -0.0004925 ], shape=(1989,), dtype=float32)
tf.Tensor(
[ 0.00282948 -0.00053326 -0.00031089 ...  0.00128379  0.00050001
  0.00055842], shape=(1989,), dtype=float32)
tf.Tensor(
[ 0.002031    0.00079555  0.00012827 ...  0.00068482 -0.00031121
 -0.00047713], shape=(1989,), dtype=float32)
tf.Tensor(
[ 9.6584909e-04  4.6185355e-04 -6.7675421e-05 ...  6.4783751e-05
  5.6384807e-04  9.0151129e-04], shape=(1989,), dtype=float32)
tf.Tensor(
[ 0.00089645  0.00013257  0.0001613  ... -0.00085959  0.00064298
  0.00114447], shape=(1989,), dtype=float32)
tf.Tensor(
[-0.0016479  -0.00085547 -0.00027578 ... -0.00022778 -0.00025267
 -0.0001

In [ ]:
print(dgcnw_21)
print(dgcnw_22)

tf.Tensor(
[[ 0.00029789  0.00201491 -0.00648069  0.01389845  0.00836645  0.00145895
  -0.02441057 -0.01098043  0.00493936 -0.0214923   0.0115817   0.01384864]
 [-0.00021403  0.01415066 -0.00927864  0.01053785  0.0033197   0.01904094
   0.00085917 -0.00910022 -0.00087449 -0.01448607 -0.01193416  0.0160376 ]
 [ 0.01957818 -0.02891149 -0.02221299  0.0107542   0.01924048 -0.01220929
   0.01106844  0.0248166  -0.01316291 -0.01150865  0.00466748  0.03123908]
 [-0.0371656   0.02621641 -0.01947146 -0.0081041   0.01467387 -0.01088396
  -0.03585733  0.01706462  0.02969152  0.00656327  0.00905009  0.00058333]
 [-0.00117717 -0.04082363 -0.03665594 -0.00242532  0.03718638 -0.03280127
   0.04231646  0.02866923 -0.01343573 -0.03498505 -0.02885667  0.03466314]
 [ 0.00043805 -0.00565056  0.0025662   0.00303786  0.00087668  0.00330156
   0.00951231  0.00957337 -0.00059335  0.0104736   0.00833223  0.00247041]
 [ 0.00087768  0.00158554  0.00020522 -0.00080314 -0.00345031 -0.0015659
  -0.00683897  0.01255

#Results

In [ ]:
model.fit(inputs, labels, epochs = 1, batch_size = 1, shuffle=False, steps_per_epoch=1)

In [ ]:
print((weights_copy_9-model.weights[9]))
print(deriv*0.01)
#print(dW/(weights_copy_6-model.weights[6]))
#print(dU/(weights_copy_7-model.weights[7]))

tf.Tensor(
[[-3.7252903e-08 -3.7252903e-08 -3.7252903e-08 ... -3.7252903e-08
  -3.7252903e-08 -3.7252903e-08]
 [-1.4901161e-08 -1.5366822e-08 -1.4901161e-08 ... -1.5366822e-08
  -1.4901161e-08 -1.4901161e-08]
 [-1.6950071e-07 -1.7136335e-07 -1.6973354e-07 ... -1.7136335e-07
  -1.7136335e-07 -1.6950071e-07]
 ...
 [-2.6077032e-08 -2.6077032e-08 -2.6077032e-08 ... -2.6077032e-08
  -2.6077032e-08 -2.6077032e-08]
 [-2.2351742e-07 -2.2165477e-07 -2.2165477e-07 ... -2.2165477e-07
  -2.2165477e-07 -2.2351742e-07]
 [-6.3329935e-08 -6.3329935e-08 -6.3329935e-08 ... -6.3329935e-08
  -6.3329935e-08 -6.3329935e-08]], shape=(16, 1989), dtype=float32)
tf.Tensor(
[[ 7.5924072e-05  1.9803758e-04  3.9285522e-05 ... -2.2464973e-04
  -3.0350615e-05  6.6195767e-05]
 [ 3.1659936e-05  8.2580635e-05  1.6381857e-05 ... -9.3677758e-05
  -1.2656047e-05  2.7603288e-05]
 [ 3.5003081e-04  9.1300759e-04  1.8111705e-04 ... -1.0356969e-03
  -1.3992467e-04  3.0518064e-04]
 ...
 [ 5.5405813e-05  1.4451852e-04  2.8668730

In [ ]:
print(dW*0.01)
print(dU*0.01)

tf.Tensor(
[[-8.50421316e-07  2.18730329e-05 -1.08773374e-05 ...  7.11414250e-05
   4.88464138e-07  7.12891851e-05]
 [ 1.54131885e-06  1.36737090e-05 -8.94579171e-06 ...  9.61717087e-05
   2.40909703e-05  4.58236391e-05]
 [ 4.47486946e-06 -1.35282198e-06 -2.69731936e-06 ...  1.06001804e-04
   4.84137745e-05  2.16244939e-06]
 ...
 [ 9.97445909e-07  1.45012164e-05  5.26635313e-06 ...  5.47873824e-05
  -1.45426411e-05  7.47745216e-05]
 [ 8.73622321e-06  8.83961638e-06 -1.06815032e-05 ...  1.77870010e-04
   7.45407742e-05  2.52130994e-05]
 [ 9.09938899e-06  1.26905734e-05 -1.40198144e-05 ...  2.24580450e-04
   8.63099558e-05  4.81858224e-05]], shape=(1989, 64), dtype=float32)
tf.Tensor(
[[ 8.4114705e-05 -1.7814485e-04  1.4879903e-05 ...  1.0237008e-03
   7.8856875e-04 -6.3892466e-04]
 [ 9.9908073e-05 -2.1854957e-04  1.3986954e-05 ...  1.1305022e-03
   9.2240155e-04 -8.0861500e-04]
 [ 6.4094071e-05 -1.5188931e-04  8.7321014e-06 ...  6.5419020e-04
   5.8324379e-04 -5.7083281e-04]
 ...
 [ 8.1

In [ ]:
print((weights_copy-model.weights[9])/deriv)

tf.Tensor(
[[-1.2853586e-06 -3.1667834e-05 -1.4870640e-06 ... -1.8878674e-06
   6.6301300e-06 -1.2896087e-06]
 [-1.2646431e-06 -3.1157459e-05 -1.4630975e-06 ... -1.8574416e-06
   6.5232753e-06 -1.2688247e-06]
 [-1.2636127e-06 -3.1132076e-05 -1.4619056e-06 ... -1.8559282e-06
   6.5179606e-06 -1.2677910e-06]
 ...
 [-1.2681522e-06 -3.1243915e-05 -1.4671574e-06 ... -1.8625955e-06
   6.5413760e-06 -1.2723455e-06]
 [-1.2162900e-06 -2.9966170e-05 -1.4071568e-06 ... -1.7864231e-06
   6.2738609e-06 -1.2203118e-06]
 [-1.2673444e-06 -3.1498712e-05 -1.4619231e-06 ... -1.8614091e-06
   6.5420022e-06 -1.2678062e-06]], shape=(16, 1989), dtype=float32)


In [ ]:
print(np.sum((model.weights[9]-weights_copy)/deriv)/(1989*16))

-2.4976053351490507e-06


#Gradient Tape

In [ ]:
mse_error = tf.keras.losses.MeanSquaredError()
with tf.GradientTape() as g:
  c=cinitial
  h=hinitial
  cache=[]
  inputs_np=[]
  cache.append((h, c, 0, 0, 0, 0, 0))
  g.watch(h_graph1)

  a1=tf.transpose(inputs, [0, 2, 1])
  neighbours1=tf.Variable(K.dot(a1, tf.Variable(model.weights[2])))
  
  h_graph1 = tf.Variable(tf.transpose(neighbours1, [0, 2, 1]))
  output11 = K.dot(h_graph1, model.weights[0])
  
  output11 += model.weights[1]
  output12 = tf.keras.activations.relu(output11)
  
  a=tf.transpose(output12, [0, 2, 1])
  neighbours2=K.dot(a, model.weights[5])
  h_graph2 = tf.transpose(neighbours2, [0, 2, 1])
  
  output21 = K.dot(h_graph2, model.weights[3])
  output21 += model.weights[4]
  
  output22 = tf.keras.activations.relu(output21)
  output23 = tf.transpose(output22, [0, 2, 1])
  output24 = tf.squeeze(output23)

  for i in range(16):
    inp=tf.reshape(output24[i], [1, 1989])
    z = K.dot(inp, model.weights[6])
    z += K.dot(h, model.weights[7])
    z = K.bias_add(z, model.weights[8])
    z0, z1, z2, z3 = array_ops.split(z, 4, axis=1)

    i = tf.sigmoid(z0)
    f = tf.sigmoid(z1)
    c = f * c + i * tf.tanh(z2)
    o = tf.sigmoid(z3)
    h = o * tf.tanh(c)
    cache.append((h, c, i, f, tf.tanh(z2), o, inp))
  
  x = K.dot(h, model.weights[9])
  y_= tf.sigmoid(x)
  loss = pow((labels-y_),2)

dy_dx = g.gradient(loss, h_graph1)

In [ ]:
print(dy_dx)

None


In [ ]:
print(tf.transpose(tf.squeeze(dy_dx))/dgcn_21)

tf.Tensor(
[[0.9999998  0.9999996  1.         ... 0.9999966  0.99999946 1.0000014 ]
 [0.99999946 0.9999996  1.         ... 0.99999136 1.         1.0000026 ]
 [1.         1.0000004  1.         ... 1.0000073  0.9999998  1.0000006 ]
 ...
 [0.9999998  0.9999995  1.         ... 1.0000013  0.9999993  1.0000008 ]
 [0.99999976 0.99999875 0.9999999  ... 0.999997   1.0000018  1.0000008 ]
 [0.9999997  0.99999994 0.99999994 ... 0.99999946 0.9999997  1.0000008 ]], shape=(12, 1989), dtype=float32)


In [ ]:
print(dy_dx)

In [ ]:
print(tf.transpose(tf.squeeze(dy_dx))/tf.transpose(dgcn_12))

tf.Tensor(
[[0.9999997  0.9999999  0.99999976 ... 0.99999964 1.0000005  0.9999991 ]
 [0.9999998  1.         0.99999994 ... 0.99999815 1.0000001  1.0000013 ]
 [0.9999999  1.         0.9999999  ... 0.99999994 1.         1.0000002 ]
 ...
 [0.9999955  1.0000055  1.0000014  ... 0.9999976  1.0000048  1.0010421 ]
 [0.9999995  0.9999999  0.9999992  ... 0.99999994 0.9999998  1.0000002 ]
 [0.9999996  1.0000002  1.0000005  ... 1.0000008  1.         1.0000008 ]], shape=(1989, 16), dtype=float32)


In [ ]:
print(output22)

tf.Tensor(
[[[0.         0.37457854 0.         ... 0.06590317 0.3412729  0.08075508]
  [0.         0.26653492 0.         ... 0.00142281 0.16358149 0.        ]
  [0.         0.08511838 0.         ... 0.03804955 0.         0.        ]
  ...
  [0.         0.         0.53635997 ... 0.         0.4410924  0.        ]
  [0.         0.30577073 0.         ... 0.         0.         0.        ]
  [0.         0.35701543 0.         ... 0.         0.         0.        ]]], shape=(1, 1989, 16), dtype=float32)


In [ ]:
print(dy_dx)

tf.Tensor(
[[[-0.0000000e+00 -4.5261451e-04 -0.0000000e+00 ... -8.0157854e-03
   -5.8270320e-03  3.9514210e-03]
  [-0.0000000e+00  2.8812414e-04  0.0000000e+00 ... -1.6731335e-02
   -2.2569312e-02 -0.0000000e+00]
  [-0.0000000e+00 -6.3295208e-04 -0.0000000e+00 ... -1.5918117e-02
   -0.0000000e+00 -0.0000000e+00]
  ...
  [ 0.0000000e+00 -0.0000000e+00  9.3821516e-05 ...  0.0000000e+00
    8.2529699e-03  0.0000000e+00]
  [-0.0000000e+00  3.3472123e-04  0.0000000e+00 ... -0.0000000e+00
   -0.0000000e+00 -0.0000000e+00]
  [ 0.0000000e+00  5.8892986e-04  0.0000000e+00 ...  0.0000000e+00
   -0.0000000e+00  0.0000000e+00]]], shape=(1, 1989, 16), dtype=float32)


In [ ]:
print(drelu_1)

tf.Tensor(
[[ 1.0333067e-06  1.1643270e-04  6.7490706e-04 ... -0.0000000e+00
   8.0665103e-07 -0.0000000e+00]
 [ 4.5261445e-04 -0.0000000e+00  6.3295214e-04 ...  6.8776171e-05
  -0.0000000e+00 -0.0000000e+00]
 [ 4.5307141e-04 -0.0000000e+00  7.9013000e-04 ... -0.0000000e+00
  -0.0000000e+00 -0.0000000e+00]
 ...
 [ 8.0157844e-03  1.6731335e-02  1.5918121e-02 ... -0.0000000e+00
   1.7391814e-02 -0.0000000e+00]
 [ 5.8270292e-03  2.2569314e-02  2.3980100e-02 ... -0.0000000e+00
   3.2384679e-02  3.0975096e-04]
 [-0.0000000e+00  4.1988965e-02  3.3135101e-02 ... -0.0000000e+00
   6.1077211e-02 -0.0000000e+00]], shape=(16, 1989), dtype=float32)


In [ ]:
print(weights_copy-model.weights[6])
model.fit(inputs, labels, epochs = 1, batch_size = 1, shuffle=False, steps_per_epoch=1)
print(weights_copy-model.weights[6])

tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(1989, 64), dtype=float32)
1/1 [==============================] - 1s 659ms/step - loss: 1.2634 - MSE: 1.2634
tf.Tensor(
[[-3.7252903e-09  0.0000000e+00  0.0000000e+00 ...  1.8626451e-09
   0.0000000e+00  0.0000000e+00]
 [-3.7252903e-09 -1.8626451e-09  1.8626451e-09 ...  6.0535967e-09
  -1.8626451e-09  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  3.2596290e-09
  -4.6566129e-10  0.0000000e+00]
 ...
 [-1.8626451e-09 -3.7252903e-09  7.4505806e-09 ...  7.4505806e-09
  -3.7252903e-09  0.0000000e+00]
 [-3.7252903e-09 -3.7252903e-09  7.4505806e-09 ...  7.4505806e-09
  -3.7252903e-09  0.0000000e+00]
 [-5.5879354e-09 -3.7252903e-09  7.4505806e-09 ...  6.5192580e-09
  -3.7252903e-09  0.0000000e+00]], shape=(1989, 64), dtype=float32)


In [ ]:
print(tf.transpose(tf.squeeze(dy_dx))/drelu_1) #rememeber 0/0 divisions are here

tf.Tensor(
[[-0.         -0.         -0.         ...         nan -0.
          nan]
 [-1.0000001         -inf -0.9999999  ... -0.                -inf
         -inf]
 [-0.                 nan -0.         ...        -inf         nan
          nan]
 ...
 [-1.0000001  -1.         -0.99999976 ...         nan -0.
          nan]
 [-1.0000005  -0.99999994 -0.         ...        -inf -0.
  -0.        ]
 [       -inf -0.         -0.         ...         nan -0.
          nan]], shape=(16, 1989), dtype=float32)


In [ ]:
print(tf.transpose(tf.squeeze(dy_dx)))
print(drelu_1) #rememeber 0/0 divisions are here

tf.Tensor(
[[-0.0000000e+00 -0.0000000e+00 -0.0000000e+00 ...  0.0000000e+00
  -0.0000000e+00  0.0000000e+00]
 [-4.5261451e-04  2.8812414e-04 -6.3295208e-04 ... -0.0000000e+00
   3.3472123e-04  5.8892986e-04]
 [-0.0000000e+00  0.0000000e+00 -0.0000000e+00 ...  9.3821516e-05
   0.0000000e+00  0.0000000e+00]
 ...
 [-8.0157854e-03 -1.6731335e-02 -1.5918117e-02 ...  0.0000000e+00
  -0.0000000e+00  0.0000000e+00]
 [-5.8270320e-03 -2.2569312e-02 -0.0000000e+00 ...  8.2529699e-03
  -0.0000000e+00 -0.0000000e+00]
 [ 3.9514210e-03 -0.0000000e+00 -0.0000000e+00 ...  0.0000000e+00
  -0.0000000e+00  0.0000000e+00]], shape=(16, 1989), dtype=float32)
tf.Tensor(
[[ 1.0333067e-06  1.1643270e-04  6.7490706e-04 ... -0.0000000e+00
   8.0665103e-07 -0.0000000e+00]
 [ 4.5261445e-04 -0.0000000e+00  6.3295214e-04 ...  6.8776171e-05
  -0.0000000e+00 -0.0000000e+00]
 [ 4.5307141e-04 -0.0000000e+00  7.9013000e-04 ... -0.0000000e+00
  -0.0000000e+00 -0.0000000e+00]
 ...
 [ 8.0157844e-03  1.6731335e-02  1.5918121

In [ ]:
print(dy_dx/dX)

tf.Tensor(
[[-0.9988541  -1.000276   -0.99852455 ... -0.99970657 -0.99959946
  -0.9994376 ]
 [-0.9989385  -1.0003607  -1.0025564  ... -0.9997082  -0.99983555
  -0.9995008 ]
 [-0.99898726 -1.0005107  -1.001682   ... -0.9997343  -0.9999698
  -0.999733  ]
 ...
 [-1.0027388  -1.0010589  -0.9944255  ... -1.0001374  -1.000461
  -1.0006317 ]
 [-1.0010654  -1.0015473  -0.9988224  ... -0.9998197  -1.0006789
  -1.0006393 ]
 [-1.0070692  -1.0013258  -0.92062336 ... -0.9952428  -1.0010668
  -1.0008498 ]], shape=(16, 1989), dtype=float32)


In [ ]:
print(dW.numpy())

[[ 1.8307447e-02 -6.9423462e-04  1.2860420e-02 ... -1.5051897e-02
   9.6502453e-03 -1.8560666e-03]
 [ 2.8206749e-02 -6.5157306e-05  1.7962879e-02 ... -2.2978727e-02
   1.4460633e-02 -2.8578993e-03]
 [ 3.3633634e-02  9.0395333e-05  1.9635409e-02 ... -2.7379684e-02
   1.6955512e-02 -3.3869424e-03]
 ...
 [ 2.1730740e-02 -1.0295080e-04 -4.7738934e-03 ... -1.5753683e-02
   1.2310137e-02 -1.3089129e-03]
 [ 4.4622585e-02 -9.3272189e-04  3.2643367e-02 ... -3.7607774e-02
   2.2956241e-02 -4.7671837e-03]
 [ 5.0856035e-02 -7.9072639e-04  3.2364108e-02 ... -4.1890424e-02
   2.5904674e-02 -5.1574740e-03]]


In [ ]:
print(dy_dx/deriv)

tf.Tensor(
[[-1.         -1.         -0.9999999  ... -1.         -1.
  -1.        ]
 [-1.         -1.         -1.         ... -1.         -1.
  -1.        ]
 [-1.         -1.         -0.9999999  ... -1.         -1.
  -1.        ]
 ...
 [-1.         -1.         -1.         ... -1.         -1.
  -1.        ]
 [-1.         -1.         -1.         ... -1.         -1.
  -1.        ]
 [-1.         -1.         -0.99999994 ... -1.         -1.
  -1.        ]], shape=(16, 1989), dtype=float32)


In [ ]:
print(0.01*deriv)

tf.Tensor(
[[ 5.9951970e-04 -4.9307797e-04 -3.4792948e-04 ... -6.1345403e-04
  -6.6194311e-04 -9.3605410e-04]
 [ 1.5699134e-04 -1.2911833e-04 -9.1109461e-05 ... -1.6064021e-04
  -1.7333764e-04 -2.4511688e-04]
 [ 1.5020660e-04 -1.2353816e-04 -8.7171946e-05 ... -1.5369775e-04
  -1.6584643e-04 -2.3452358e-04]
 ...
 [ 7.0560916e-04 -5.8033183e-04 -4.0949817e-04 ... -7.2200922e-04
  -7.7907881e-04 -1.1016958e-03]
 [ 6.2830187e-04 -5.1674998e-04 -3.6463307e-04 ... -6.4290507e-04
  -6.9372205e-04 -9.8099280e-04]
 [-2.0943006e-04  1.7224680e-04  1.2154211e-04 ...  2.1429773e-04
   2.3123639e-04  3.2699155e-04]], shape=(16, 1989), dtype=float32)


In [ ]:
"""fixed_layers_inputs=[]
a=tf.transpose(inputs, [0, 2, 1])
neighbours=K.dot(a, model.weights[2])

fixed_layers_inputs.append(a)
h_graph = tf.transpose(neighbours, [0, 2, 1])
output = K.dot(h_graph, model.weights[0])

fixed_layers_inputs.append(h_graph)
output += model.weights[1]
output = tf.keras.activations.relu(output)


a=output

neighbours=K.dot(tf.transpose(a, [0, 2, 1]), model.weights[5])
fixed_layers_inputs.append(a)
h_graph = tf.transpose(neighbours, [0, 2, 1])
output = K.dot(h_graph, model.weights[3])
fixed_layers_inputs.append(h_graph)
output += model.weights[4]
output = tf.keras.activations.relu(output)
output = tf.transpose(output, [0, 2, 1])

c=cinitial
h=hinitial
cache=[]
inputs_np=[]
cache.append((h, c, 0, 0, 0, 0, 0))
for i in range(16):
    inp=tf.constant([output[0][i].numpy()])
    inputs_np.append(inp)
    z = K.dot(inp, model.weights[6])
    z += K.dot(h, model.weights[7])
    z = K.bias_add(z, model.weights[8])
    z0, z1, z2, z3 = array_ops.split(z, 4, axis=1)

    i = tf.sigmoid(z0)
    f = tf.sigmoid(z1)
    c = f * c + i * tf.tanh(z2)
    o = tf.sigmoid(z3)
    h = o * tf.tanh(c)

    cache.append((h, c, i, f, tf.tanh(z2), o, inp))"""

In [ ]:
print(dy_dx)

tf.Tensor(
[[ 0.00157923  0.00157923  0.00157923 ...  0.00157923  0.00157923
   0.00157923]
 [ 0.11346778  0.11346778  0.11346778 ...  0.11346778  0.11346778
   0.11346778]
 [ 0.10672068  0.10672068  0.10672068 ...  0.10672068  0.10672068
   0.10672068]
 ...
 [ 0.17252511  0.17252511  0.17252511 ...  0.17252511  0.17252511
   0.17252511]
 [-0.3154959  -0.3154959  -0.3154959  ... -0.3154959  -0.3154959
  -0.3154959 ]
 [-0.10146282 -0.10146282 -0.10146282 ... -0.10146282 -0.10146282
  -0.10146282]], shape=(16, 1989), dtype=float32)


#Saving files

In [ ]:
values=[]
for i in inputs.numpy():
  for j in i:
    for k in j:
      values.append(str(k)+'\n')
print(len(values))

f=open('inputs.inp','w')
for i in values:
  f.write(i)
f.close()

files.download('inputs.inp')

23868


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(labels)

tf.Tensor(
[[-1.4258932   0.13763477  1.3488818  ...  0.26632673 -0.7557638
   0.04721673]], shape=(1, 1989), dtype=float32)


In [ ]:
values=[]
for i in labels.numpy():
  for j in i:
      values.append(str(j)+'\n')

f=open('labels.inp','w')
for i in values:
  f.write(i)
f.close()

files.download('labels.inp')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
values=[]
for i in hinitial.numpy():
  for j in i:
      values.append(str(j)+'\n')
print(len(values))

f=open('hinitial.inp','w')
for i in values:
  f.write(i)
f.close()

files.download('hinitial.inp')

values=[]
for i in cinitial.numpy():
  for j in i:
      values.append(str(j)+'\n')
print(len(values))

f=open('cinitial.inp','w')
for i in values:
  f.write(i)
f.close()

files.download('cinitial.inp')

16


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

16


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
#files.download('gcn_inp.inp')
files.download('A.inp')
files.download('kernel.inp')
files.download('bias.inp')
files.download('A2.inp')
files.download('kernel2.inp')
files.download('bias2.inp')
files.download('kernel3.inp')
files.download('bias3.inp')
files.download('recurrent_kernel3.inp')
files.download('kernel4.inp')
files.download('bias4.inp')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
f=open('A.inp')
lines=f.readlines()
print(len(lines))

3956121


In [ ]:
for i in range(len(model.weights)):
  print(i, model.weights[i].name, model.weights[i].shape)

0 fixed_adjacency_graph_convolution_2/kernel:0 (12, 4)
1 fixed_adjacency_graph_convolution_2/bias:0 (1989, 1)
2 fixed_adjacency_graph_convolution_2/A:0 (1989, 1989)
3 fixed_adjacency_graph_convolution_3/kernel:0 (4, 4)
4 fixed_adjacency_graph_convolution_3/bias:0 (1989, 1)
5 fixed_adjacency_graph_convolution_3/A:0 (1989, 1989)
6 lstm_2/lstm_cell_2/kernel:0 (1989, 16)
7 lstm_2/lstm_cell_2/recurrent_kernel:0 (4, 16)
8 lstm_2/lstm_cell_2/bias:0 (16,)
9 lstm_3/lstm_cell_3/kernel:0 (4, 16)
10 lstm_3/lstm_cell_3/recurrent_kernel:0 (4, 16)
11 lstm_3/lstm_cell_3/bias:0 (16,)
12 dense_1/kernel:0 (4, 1989)
13 dense_1/bias:0 (1989,)


In [ ]:
#horribly written code that should be dumped in a function
for item in range(3):
  n=model.weights[item].name
  print(n[(n.rindex('/')+1):-2]+'.inp')
  w=model.weights[item].numpy()
  f=open(n[(n.rindex('/')+1):-2]+'.inp','w')
  print(model.weights[item].shape)
  values=[]
  for i in w:
    for j in i:
        values.append(str(j)+'\n')
  for i in values:
      f.write(i)
  f.close()

for item in range(3,6):
  n=model.weights[item].name
  print(n[(n.rindex('/')+1):-2]+'2.inp')
  w=model.weights[item].numpy()
  f=open(n[(n.rindex('/')+1):-2]+'2.inp','w')
  print(model.weights[item].shape)
  values=[]
  for i in w:
    for j in i:
        values.append(str(j)+'\n')
  for i in values:
      f.write(i)
  f.close()

for item in range(6,8):
  n=model.weights[item].name
  print(n[(n.rindex('/')+1):-2]+'3.inp')
  w=model.weights[item].numpy()
  f=open(n[(n.rindex('/')+1):-2]+'3.inp','w')
  print(model.weights[item].shape)
  values=[]
  for i in w:
    for j in i:
        values.append(str(j)+'\n')
  for i in values:
      f.write(i)
  f.close()

n=model.weights[8].name
print(n[(n.rindex('/')+1):-2]+'3.inp')
w=model.weights[8].numpy()
f=open(n[(n.rindex('/')+1):-2]+'3.inp','w')
print(model.weights[8].shape)
values=[]
for i in w:
  values.append(str(i)+'\n')
for i in values:
  f.write(i)
f.close()

n=model.weights[10].name
print(n[(n.rindex('/')+1):-2]+'4.inp')
w=model.weights[10].numpy()
f=open(n[(n.rindex('/')+1):-2]+'4.inp','w')
print(model.weights[10].shape)
values=[]
for i in w:
  values.append(str(i)+'\n')
for i in values:
  f.write(i)
f.close()

n=model.weights[9].name
print(n[(n.rindex('/')+1):-2]+'4.inp')
w=model.weights[9].numpy()
f=open(n[(n.rindex('/')+1):-2]+'4.inp','w')
print(model.weights[9].shape)
values=[]
for i in w:
  for j in i:
    values.append(str(j)+'\n')
for i in values:
  f.write(i)
f.close()

kernel.inp
(12, 16)
bias.inp
(1989, 1)
A.inp
(1989, 1989)
kernel2.inp
(16, 16)
bias2.inp
(1989, 1)
A2.inp
(1989, 1989)
kernel3.inp
(1989, 64)
recurrent_kernel3.inp
(16, 64)
bias3.inp
(64,)
bias4.inp
(1989,)
kernel4.inp
(16, 1989)


In [ ]:
"""#horribly written code that should be dumped in a function
for item in range(3):
  n=model.weights[item].name
  print(n[(n.rindex('/')+1):-2]+'.inp')
  w=model.weights[item].numpy()
  f=open(n[(n.rindex('/')+1):-2]+'.inp','w')
  print(model.weights[item].shape)
  values=[]
  for i in w:
    for j in i:
        values.append(str(j)+'\n')
  for i in values:
      f.write(i)
  f.close()

print()

for item in range(3,6):
  n=model.weights[item].name
  print(n[(n.rindex('/')+1):-2]+'2.inp')
  w=model.weights[item].numpy()
  f=open(n[(n.rindex('/')+1):-2]+'2.inp','w')
  print(model.weights[item].shape)
  values=[]
  for i in w:
    for j in i:
        values.append(str(j)+'\n')
  for i in values:
      f.write(i)
  f.close()

print()

for item in range(6,8):
  n=model.weights[item].name
  print(n[(n.rindex('/')+1):-2]+'3.inp')
  w=model.weights[item].numpy()
  f=open(n[(n.rindex('/')+1):-2]+'3.inp','w')
  print(model.weights[item].shape)
  values=[]
  for i in w:
    for j in i:
        values.append(str(j)+'\n')
  for i in values:
      f.write(i)
  f.close()

print()

for item in range(9,11):
  n=model.weights[item].name
  print(n[(n.rindex('/')+1):-2]+'4.inp')
  w=model.weights[item].numpy()
  f=open(n[(n.rindex('/')+1):-2]+'4.inp','w')
  print(model.weights[item].shape)
  values=[]
  for i in w:
    for j in i:
        values.append(str(j)+'\n')
  for i in values:
      f.write(i)
  f.close()

print()

n=model.weights[8].name
print(n[(n.rindex('/')+1):-2]+'3.inp')
w=model.weights[8].numpy()
f=open(n[(n.rindex('/')+1):-2]+'3.inp','w')
print(model.weights[8].shape)
values=[]
for i in w:
  values.append(str(i)+'\n')
for i in values:
  f.write(i)
f.close()

n=model.weights[11].name
print(n[(n.rindex('/')+1):-2]+'4.inp')
w=model.weights[11].numpy()
f=open(n[(n.rindex('/')+1):-2]+'4.inp','w')
print(model.weights[11].shape)
values=[]
for i in w:
  values.append(str(i)+'\n')
for i in values:
  f.write(i)
f.close()

print()

n=model.weights[13].name
print(n[(n.rindex('/')+1):-2]+'5.inp')
w=model.weights[13].numpy()
f=open(n[(n.rindex('/')+1):-2]+'5.inp','w')
print(model.weights[13].shape)
values=[]
for i in w:
  values.append(str(i)+'\n')
for i in values:
  f.write(i)
f.close()

n=model.weights[12].name
print(n[(n.rindex('/')+1):-2]+'5.inp')
w=model.weights[12].numpy()
f=open(n[(n.rindex('/')+1):-2]+'5.inp','w')
print(model.weights[12].shape)
values=[]
for i in w:
  for j in i:
    values.append(str(j)+'\n')
for i in values:
  f.write(i)
f.close()"""